In [1]:
from ultralytics import YOLO
import json
import os
import shutil
import random
import cv2
import matplotlib.pyplot as plt

In [2]:
json_path = "annotations.json"
images_dir = "images"
labels_dir = "labels"
data_yaml = "data.yaml"
train_dir = "dataset/train"
val_dir = "dataset/val"
os.makedirs(train_dir + "/images", exist_ok=True)
os.makedirs(train_dir + "/labels", exist_ok=True)
os.makedirs(val_dir + "/images", exist_ok=True)
os.makedirs(val_dir + "/labels", exist_ok=True)

In [3]:
# Load JSON annotation file
with open(json_path, "r") as f:
    data = json.load(f)

file_names = data["train"]["file_names"]
rois_list = data["train"]["rois_list"]
occupancy_list = data["train"]["occupancy_list"]

# Split dataset (80% train, 20% val)
data_pairs = list(zip(file_names, rois_list, occupancy_list))
random.shuffle(data_pairs)
split_idx = int(0.8 * len(data_pairs))
train_data = data_pairs[:split_idx]
val_data = data_pairs[split_idx:]

In [4]:
# Function to process and save labels
def save_labels(file_name, rois, occupancy, split):
    label_path = f"dataset/{split}/labels/" + file_name.replace(".JPG", ".txt")
    img_path = os.path.join(images_dir, file_name)
    shutil.copy(img_path, f"dataset/{split}/images/")
    
    with open(label_path, "w") as lf:
        for obj, occupied in zip(rois, occupancy):
            x_values = [p[0] for p in obj]
            y_values = [p[1] for p in obj]
            x_center = sum(x_values) / len(x_values)
            y_center = sum(y_values) / len(y_values)
            width = max(x_values) - min(x_values)
            height = max(y_values) - min(y_values)
            class_id = 0 if occupied else 1
            lf.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Process all data
for file_name, rois, occupancy in train_data:
    save_labels(file_name, rois, occupancy, "train")
for file_name, rois, occupancy in val_data:
    save_labels(file_name, rois, occupancy, "val")

In [5]:
# Write data.yaml
with open(data_yaml, "w") as f:
    f.write("train: dataset/train\n")
    f.write("val: dataset/val\n")
    f.write("nc: 2\n")
    f.write("names: ['car', 'empty']\n")

In [6]:
# Load model
model = YOLO("yolo11n.pt")

# Train model
results = model.train(
    data=data_yaml, 
    epochs=5, 
    imgsz=640,
    batch=16,
    optimizer='Adam'
)

# Change path in C:\Users\[user]\AppData\Roaming\Ultralytics\settings.json to where the "ML Model" folder is stored on your computer

Ultralytics 8.3.81  Python-3.13.2 torch-2.6.0+cpu CPU (Intel Core(TM) i7-8700K 3.70GHz)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train25, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, lin

train: Scanning C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\dataset\train\labels... 220 images, 0 backgrounds, 0 corrupt: 100%|██████████| 220/220 [00:00<00:00, 257.36it/s]

train: New cache created: C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\dataset\train\labels.cache



val: Scanning C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\dataset\val\labels... 83 images, 0 backgrounds, 0 corrupt: 100%|██████████| 83/83 [00:00<00:00, 453.89it/s]


val: New cache created: C:\Users\Aman.DESKTOP-VEB4T1B\OneDrive\Documents\Rutgers\Capstone\ece-capstone\ML Model\dataset\val\labels.cache
Plotting labels to runs\detect\train25\labels.jpg... 
optimizer: Adam(lr=0.01, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train25
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

WARNING  NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  67%|██████▋   | 2/3 [00:17<00:08,  8.52s/it]

WARNING  NMS time limit 2.950s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:22<00:00,  7.62s/it]


                   all         83       2579      0.201      0.146      0.123     0.0499
                   car         83       1314      0.262      0.235      0.196     0.0787
                 empty         77       1265       0.14     0.0569     0.0505      0.021
Speed: 1.9ms preprocess, 82.7ms inference, 0.0ms loss, 127.1ms postprocess per image
Results saved to runs\detect\train25


In [9]:
# Visualize sample images
def visualize_samples(split, num=5):
    img_dir = f"dataset/{split}/images"
    label_dir = f"dataset/{split}/labels"

    sample_images = random.sample(os.listdir(img_dir), num)
    
    for img_file in sample_images:
        img_path = os.path.join(img_dir, img_file)
        label_path = os.path.join(label_dir, img_file.replace(".JPG", ".txt"))

        if not os.path.exists(img_path) or not os.path.exists(label_path):
            print(f"Skipping {img_file}, missing label or image file!")
            continue

        img = cv2.imread(img_path)
        if img is None:
            print(f"Error: Could not load image {img_path}")
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        h_img, w_img, _ = img.shape

        print(f"Processing {img_file} | Shape: {img.shape}")

        with open(label_path, "r") as lf:
            labels = lf.readlines()
            print(f"Label contents for {img_file}:\n{labels}")

            for label in labels:
                class_id, x, y, w, h = map(float, label.split())

                # Convert YOLO format (normalized) to pixel coordinates
                x1 = int((x - w / 2) * w_img)
                y1 = int((y - h / 2) * h_img)
                x2 = int((x + w / 2) * w_img)
                y2 = int((y + h / 2) * h_img)

                color = (255, 0, 0) if class_id == 0 else (0, 255, 0)
                label_name = "Car" if class_id == 0 else "Empty"

                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(img, label_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        plt.axis("off")
        plt.show(block=True)  # Force display

# Run and debug visualization
visualize_samples("train")

Processing GOPR0049.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR0049.JPG:
['1 0.7893126874999999 0.794413 0.2617702500000001 0.34651599999999994\n', '0 0.6459510625 0.6704331666666666 0.30885124999999997 0.34526066666666666\n', '0 0.5221280625 0.5593220833333333 0.27118650000000005 0.259887\n', '0 0.4357345 0.4789704166666666 0.25047100000000005 0.21594466666666667\n', '1 0.3667608125 0.41494033333333336 0.22222225 0.17702466666666666\n', '0 0.31156887499999997 0.36471983333333335 0.19578299999999998 0.14216866666666667\n', '1 0.266689375 0.32476008333333334 0.16807224999999998 0.12048200000000003\n', '1 0.234008625 0.294238 0.15060224999999997 0.10923699999999995\n', '1 0.207752625 0.2698534166666667 0.13634949999999998 0.094779\n', '0 0.18442425 0.250235 0.12325000000000001 0.08600000000000002\n', '0 0.16667425 0.23448500000000003 0.10674999999999998 0.07266666666666666\n', '0 0.15304925 0.2219016666666667 0.09575 0.06766666666666668\n', '0 0.14104925000000001 0.2108183333333

<Figure size 600x600 with 1 Axes>

Processing GOPR6770.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR6770.JPG:
['0 0.9358339375 0.7797934166666667 0.10666675000000003 0.3004446666666666\n', '1 0.8655839375 0.7047934166666667 0.12766650000000002 0.28755566666666665\n', '0 0.7890005625 0.6239045833333333 0.1523332500000001 0.2653333333333333\n', '1 0.70941725 0.5447934166666667 0.17066674999999998 0.2302223333333333\n', '1 0.63316725 0.4726823333333333 0.18266650000000006 0.1906666666666666\n', '0 0.5653339374999999 0.41112675 0.18066650000000006 0.15511133333333327\n', '1 0.5072505625 0.36090466666666665 0.17233324999999994 0.12488900000000003\n', '0 0.45933393749999996 0.32168233333333335 0.1606667499999999 0.10311100000000001\n', '1 0.419083875 0.28990458333333335 0.14800000000000002 0.08755566666666667\n', '1 0.9590839375 0.5986824166666667 0.06166674999999999 0.18622233333333338\n', '1 0.9010005624999999 0.5410156666666667 0.07700000000000007 0.1755553333333334\n', '1 0.8400005625 0.4819045833333333 0.094666749

<Figure size 600x600 with 1 Axes>

Processing GOPR6765.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR6765.JPG:
['0 0.31900425 0.8789686666666667 0.288 0.27493066666666666\n', '0 0.22731675 0.7739026666666667 0.24325000000000002 0.31466666666666665\n', '0 0.1825459375 0.6674026666666667 0.20083325000000002 0.254\n', '0 0.1528375625 0.5854026666666667 0.172 0.2031110000000001\n', '1 0.133171 0.5225136666666667 0.151 0.17288899999999996\n', '1 0.93922825 0.7292846666666666 0.08000000000000007 0.2713333333333333\n', '1 0.86902 0.6684513333333333 0.10650000000000004 0.2613333333333332\n', '0 0.790895 0.6027846666666666 0.13049999999999995 0.24533333333333324\n', '1 0.70852 0.537118 0.15400000000000003 0.21266666666666662\n', '0 0.631228375 0.47689574999999995 0.16166674999999997 0.18399999999999994\n', '1 0.559895 0.4255625 0.16733325000000004 0.15377800000000008\n', '1 0.4983950625 0.383118 0.16366674999999997 0.12711099999999997\n', '1 0.9426449375 0.5637846666666666 0.059333250000000004 0.15955533333333338\n', '0 0.

<Figure size 600x600 with 1 Axes>

Processing GOPR6667.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR6667.JPG:
['1 0.35612325 0.40909366666666663 0.16375 0.1353333333333333\n', '0 0.31162325 0.36942699999999995 0.14449999999999996 0.11499999999999999\n', '1 0.48074825 0.5185103333333333 0.19474999999999998 0.20733333333333337\n', '1 0.5633732499999999 0.5925103333333334 0.19324999999999998 0.24833333333333335\n', '0 0.6565494374999999 0.6745710833333334 0.18799999999999994 0.28488900000000006\n', '0 0.7499661875 0.7505710833333333 0.16166675000000008 0.29422233333333336\n', '1 0.966318 0.913622 0.07190925000000004 0.21422200000000002\n', '1 0.27732212500000003 0.33846333333333334 0.12416675000000002 0.09555566666666665\n', '1 0.2479054375 0.31028283333333334 0.12283324999999998 0.0899443333333334\n', '1 0.21094712499999999 0.27663 0.09649999999999995 0.06666666666666668\n', '0 0.19385349999999998 0.26142166666666666 0.09125 0.06283333333333335\n', '1 0.1780305625 0.24751891666666664 0.08125000000000002 0.054833333

<Figure size 600x600 with 1 Axes>

Processing GOPR6683.JPG | Shape: (3000, 4000, 3)
Label contents for GOPR6683.JPG:
['1 0.352710125 0.48362933333333336 0.14099974999999998 0.15155566666666664\n', '0 0.6228768124999999 0.54251825 0.11299999999999999 0.19200000000000006\n', '0 0.7203768749999999 0.5652959166666667 0.10466675000000003 0.19688899999999998\n', '1 0.8064601874999999 0.5850737500000001 0.10200000000000009 0.1844446666666667\n', '0 0.524626875 0.5197404166666667 0.132 0.182222\n', '0 0.43254349999999997 0.50007375 0.14300000000000002 0.16399999999999998\n', '0 0.285106 0.46846266666666664 0.13825 0.137\n', '1 0.229481 0.456796 0.12350000000000003 0.1213333333333334\n', '1 0.185751875 0.447685 0.11733349999999998 0.11199999999999999\n', '0 0.1489601875 0.4391293333333333 0.10783350000000001 0.10355566666666671\n', '0 0.1175435 0.43151833333333334 0.09916649999999999 0.09733333333333333\n', '1 0.28098999999999996 0.2319479166666667 0.06216674999999999 0.014444333333333309\n', '0 0.2981358125 0.21992025 0.0596667

<Figure size 600x600 with 1 Axes>